Count letters

In [1]:
%pip install --upgrade tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.4/770.4 kB 56.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tiktoken


In [18]:
import tiktoken

def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += len(encoding.encode(str(message)))
        num_tokens += tokens_per_message
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

In [13]:
from datasets import load_from_disk

first_twenty = load_from_disk('/home/jupyter/generative-ai/with_abstract_train_first_twenty_percent')

In [14]:
first_twenty

Dataset({
    features: ['image_file', 'id', 'caption', 'conversations', 'first_mention', 'response', 'title', 'abstract'],
    num_rows: 70404
})

In [15]:
first_twenty['response'][0]

'Question: What is the x-axis of the graph?\n\nAnswer: The x-axis of the graph represents the number of synthetic ROIs generated with DSFS, 3DMM, and SHBMM according to pose and lighting effects.\n\nQuestion: What is the y-axis of the graph?\n\nAnswer: The y-axis of the graph represents the average AUC and AUPR obtained by increasing the number of synthetic ROIs used to design SRC and SVM classifiers on the Chokepoint and COX-S2V databases.\n\nQuestion: What does the graph show?\n\nAnswer: The graph shows that the performance of still-to-video FR systems improves as the number of synthetic ROIs increases. This is because the synthetic ROIs help to capture the variations in pose and lighting that are present in the operational domain.\n\nQuestion: What are the key takeaways from the graph?\n\nAnswer: The key takeaways from the graph are that:\n\n1. The performance of still-to-video FR systems can be improved by augmenting the reference set with synthetic faces.\n2. The number of synthet

In [26]:
output_tokens = num_tokens_from_messages(first_twenty['response']) 

In [21]:
input_tokens = num_tokens_from_messages(first_twenty['conversations']) + num_tokens_from_messages(first_twenty['abstract']) + num_tokens_from_messages(first_twenty['caption'])

In [24]:
system_message = """
      You are an AI visual assistant that can analyze a graph in a scientific paper. You are provided with the OCR-extracted text, the caption of the figure, and the first paragraph that mentioned the figure.

      Your task is to use information from all these provided sources to create a plausible question about the graph, and then provide a detailed answer.

      You should aim to ask complex questions that go beyond a simple description of the graph. The answer to such questions should require understanding the graph data, and then reasoning based on background knowledge or interpretation. You should aim to provide guides and scholarly perspectives based on the graph's data and context.

      Avoid directly revealing specific details in the question. Make the question challenging in a way that the user needs to first reason about the graph data and the context they have derived from the accompanying paper.

      Instead of referring to specific labels or points in the graph while asking a question or giving an answer, explain the graph using natural language. Include details like data trends, prominent points, and relationships between different data sets visualized in the graph.

      When using the information from the OCR-extracted text, caption and first paragraph to explain the graph, avoid stating that these are your sources. Always answer as if you are directly interpreting the graph according to your AI comprehension, understanding and reasoning. Regarding the format of the given input: the following context described the image:  "from": "human", "value": contains OCR extracted text listed, and 'Figure (integer) caption of the image'."from: gpt value: contains the first paragraph in the text that mentioned the figure. Many times the OCR is missing simply because the image does not contain text. 


Other example may include OCR extracted example like the following: 
[{'from': 'human',
  'value': "OCR extracted text list, separated by ', ' : 0.8I-R >0.750.70.650.50.60.70.80.91Source variance (y)Mean target risk (RT), 0.8, I-R, >, 0.75, 0.7, 0.65, 0.5, 0.6, 0.7, 0.8, 0.9, 1, Source, variance, (y), Mean, target, risk, (RT) Fig. 6. Mean target risks for the synthetic problem setting, as a function of source variance γ.<image>\nAnalyze the image in a comprehensive and detailed manner"},
 {'from': 'gpt',
  'value': 'Weights are estimated by fitting a normal distribution to data from each domain, and computing the ratio of the target probability over the source probability of each source data point ŵ(xi) = p̂T (xi)/p̂S(xi). We compare the importanceweighted risk estimator (R̂ŵ) with its control variate counterpart (R̂β̂). We average their estimated risks over all data sets and specifically over the 10% of data sets with the largest weight variance (indicated with ”>” in the legend of Figures 6, 7 and 8). We also include validation on the labeled target samples (R̂T ) as the oracle solution. After risk estimation, the λ is selected that minimized risk. The classifier is then retrained using all source data and the selected λ, and evaluated using the target risk based on the true target labels as the final measure. This process is repeated for each data set and we report the final average as R̄T .'}]
In this case, try to pay attention to the OCR extracted text list and establish its connection to the figure caption as well as the provided paper title and abstract. The OCR provided text is from the figure itself, which you want to help a human understand, so it is important to think about how the those characters relate to the explanation as you write down question and answer pairs. But do not copy and paste the origional OCR text. The OCR text is randomly ordered and convey no explanatory meaning by itself.

Can you please think of a couple questions a college-level student may ask regarding the image being discussed, given the context, image caption and the first paragraph that mention the image as well as the entire paper's summary? Think of a couple questions that this college student may ask to first describe the image, and then ask to be walked through the image in a helpuful, textbook-level instruction format. 


One example is like this: 
[ { "from": "human", "value": ": How does the length normalization factor affect the BLEU score?\n<image>" }, { "from": "gpt", "value": ": The graph shows that the BLEU score is highest when the length normalization factor is set to 0.9. This is true for all values of the source boundary delay D." } ]

You should think like the following as a example for the input graph: Notice that the caption gives away the x and y axis of the graph. The graph is plotting length normalization factor on the x axis and BLEU on the y  axis because from 'How does the length normalization factor affect the BLEU score?' you can infer that when X affects Y, X is a independent variable and the Y which is BLEU is the dependent variable. You would then notice that the first paragraph explains what the key summary of the graph should be: the author highlights the fact that BLEU is highest at a specific length normalization factor of 0.9. 

This is the title and summary of the paper: Neural Simultaneous Speech Translation Using Alignment-Based Chunking. In simultaneous machine translation, the objective is to determine when to produce a partial translation given a continuous stream of source words, with a trade-off between latency and quality. We propose a neural machine translation (NMT) model that makes dynamic decisions when to continue feeding on input or generate output words. The model is composed of two main components: one to dynamically decide on ending a source chunk, and another that translates the consumed chunk. We train the components jointly and in a manner consistent with the inference conditions. To generate chunked training data, we propose a method that utilizes word alignment while also preserving enough context. We compare models with bidirectional and unidirectional encoders of different depths, both on real speech and text input. Our results on the IWSLT 2020 English-to-German task outperform a wait-k baseline by 2.6 to 3.7% BLEU absolute.

The demostrated response the following have question and answers. Each Question and Answer starts on a new line, and always starts with either 'Question: ' or 'Answer: ' such as the following example. You can have 2-5 question and answer pair. 

Question: Can you examplify how the BLEU score trends according to the changes in the length normalization factor as depicted on the graph?

Answer: On close analysis, we can observe that this graph demonstrates an intriguing correlation between the "Length Normalization Factor" and the "BLEU Score". It portrays a concave-upward curve, which indicates that as the length normalization factor increases, the BLEU score initially rises, reaching a maximum value, after which it starts to decrease. The maximum value appears when the length normalization factor is set to 0.9, leading to the highest BLEU score, indicating optimum performance of the Neural Machine Translation model at this specific point. The graph interestingly captures this phenomenon, shaping our understanding of how the translation performance varies with different length normalization factors.

Question: Based on the graph, what kind of impact does the length normalization factor have on the BLEU score and how can this finding be beneficial for improving translation models like in this study, "Neural Simultaneous Speech Translation Using Alignment-Based Chunking"?

Answer: As elucidated by the graph, the length normalization factor plays a significant role in fine-tuning the performance of a Neural Machine Translation (NMT) model. Essentially, the optimum translation performance, as measured by the BLEU score, is achieved when the length normalization factor is fixed at 0.9. This signifies that finding the right balance in segment length can lead to better translation results. As for the implications for the 'Alignment-Based Chunking' in the context of our study, understanding this optimum value grants us valuable insight into setting suitable chunk sizes for processing. This consequently could help us strike a balance between latency and quality, improving overall efficiency and effectiveness of our simultaneous machine translation system. Therefore, our finding and its pictorial representation in this graph provide a significant contribution to the neural translation research domain.


Other example may include OCR extracted example like the following example. In this case, try to pay attention to the OCR extracted text list and establish its connection to the figure caption as well as the provided paper title and abstract. The OCR provided text is from the figure itself, which you want to help a human understand, so it is important to think about how the those characters relate to the explanation as you write down question and answer pairs. But do not copy and paste the origional OCR text. The OCR text is randomly ordered and convey no explanatory meaning by itself.
 
[{'from': 'human',
  'value': "OCR extracted text list, separated by ', ' : 0.8I-R >0.750.70.650.50.60.70.80.91Source variance (y)Mean target risk (RT), 0.8, I-R, >, 0.75, 0.7, 0.65, 0.5, 0.6, 0.7, 0.8, 0.9, 1, Source, variance, (y), Mean, target, risk, (RT) Fig. 6. Mean target risks for the synthetic problem setting, as a function of source variance γ.<image>\nAnalyze the image in a comprehensive and detailed manner"},
 {'from': 'gpt',
  'value': 'Weights are estimated by fitting a normal distribution to data from each domain, and computing the ratio of the target probability over the source probability of each source data point ŵ(xi) = p̂T (xi)/p̂S(xi). We compare the importanceweighted risk estimator (R̂ŵ) with its control variate counterpart (R̂β̂). We average their estimated risks over all data sets and specifically over the 10% of data sets with the largest weight variance (indicated with ”>” in the legend of Figures 6, 7 and 8). We also include validation on the labeled target samples (R̂T ) as the oracle solution. After risk estimation, the λ is selected that minimized risk. The classifier is then retrained using all source data and the selected λ, and evaluated using the target risk based on the true target labels as the final measure. This process is repeated for each data set and we report the final average as R̄T .'}]
The demostrated response the following have question and answers

Question: How does the graph depict the relationship between the source variance (γ) and the mean target risk (RT) in the context of the synthetic problem setting?

Answer: The graph presents a clear depiction of how the mean target risk (RT) changes with varying source variance (γ). It appears to showcase a trend where the mean target risk decreases as the source variance increases, which suggests an inverse relationship between the two variables. This relationship is critical in the context of the synthetic problem setting, as it implies that a higher source variance could potentially lead to a lower mean target risk, thus influencing the overall performance and efficiency of the derivative interpolating subspace frameworks for nonlinear eigenvalue problems.

Question: Given the study's focus on derivative interpolating subspace frameworks for nonlinear eigenvalue problems, how does the observed relationship between source variance and mean target risk in the graph contribute to the overall findings of the research?

Answer: The observed relationship between source variance and mean target risk in the graph is instrumental in understanding the performance of the proposed subspace framework. The graph indicates that as the source variance increases, the mean target risk decreases. This could suggest that the framework performs better when dealing with higher source variance. This insight is particularly significant as it provides a valuable parameter (source variance) that can be adjusted to optimize the performance of the derivative interpolating subspace frameworks for nonlinear eigenvalue problems. Therefore, this graph not only substantiates the research's findings but also offers a practical guideline for enhancing the efficiency of the proposed framework.

Question: Based on the graph and the context of the paper, could you explain how the concept of source variance impacts the performance of the derivative interpolating subspace framework for nonlinear eigenvalue problems?

Answer: The graph provides a visual representation of the relationship between source variance and mean target risk, two critical parameters in the context of the derivative interpolating subspace framework for nonlinear eigenvalue problems. As source variance increases, the graph shows a corresponding decrease in mean target risk. This suggests that the derivative interpolating subspace framework performs more effectively when dealing with higher source variance. In the context of nonlinear eigenvalue problems, this could mean that the framework is particularly adept at handling complex problems with a high degree of variance. This insight is significant as it not only validates the effectiveness of the proposed framework but also highlights its potential for tackling complex, high-variance nonlinear eigenvalue problems.
"""



system_message_token_count = num_tokens_from_messages([system_message,])


In [22]:
len(first_twenty)

70404

In [27]:
total_input_tokens = input_tokens + system_message_token_count * len(first_twenty)

total_output = output_tokens

print('input token count', total_input_tokens)

print('output tokens', total_output)

input token count 249334421
output tokens 15492050


In [36]:
def calculate_gpt_cost(input_token_count, output_token_count, model = 'gpt-4'):
    if model == 'gpt-4':
        return 0.03*(input_token_count//1000) + 0.006*(output_token_count//1000)
    elif model == 'gpt-3.5-turbo':
        return 0.0015*(input_token_count//1000) + 0.002*(output_token_count//1000)
    else:
        return 0.0005*(input_token_count + output_token_count) // 1000

calculate_gpt_cost(total_input_tokens, total_output, model='palm')

132.0

In [34]:
mean_output_token = (total_output) / len(first_twenty)
mean_input = input_tokens // len(first_twenty)
mean_input_with_system_message = mean_input + system_message_token_count
mean_output_token, mean_input, mean_input_with_system_message

(220.04502585080394, 774, 3541)

In [37]:
from datasets import load_dataset
compare_to_llava = load_dataset('liuhaotian/LLaVA-Instruct-150K', split='train')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/jupyter/.cache/huggingface/datasets/liuhaotian___json/liuhaotian--LLaVA-Instruct-150K-4c8b43235748f679/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


In [38]:
llava_total_output_tokens = num_tokens_from_messages(compare_to_llava['conversations'])

In [39]:
average_llava_token_length = llava_total_output_tokens // len(compare_to_llava['conversations'])

print('total llava output tokens', llava_total_output_tokens)
print('average llava output tokens', average_llava_token_length)

total llava output tokens 109185835
average llava output tokens 276


In [48]:
prev_single_turn = load_dataset('alexshengzhili/SciCapInstructed410K', split='train')
prev_single_turn_total = num_tokens_from_messages(prev_single_turn['response'])

prev_single_turn_average = prev_single_turn_total // len(prev_single_turn['response'])

print('total previous output tokens', prev_single_turn_total)
print('average previous output tokens', prev_single_turn_average)


Found cached dataset parquet (/home/jupyter/.cache/huggingface/datasets/alexshengzhili___parquet/alexshengzhili--SciCapInstructed410K-f89dbc60ba315149/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


total previous output tokens 31959233
average previous output tokens 90


In [47]:
prev_single_turn['response'][0]

'Question: The graph shows the performance of still-to-video FR systems versus the number of synthetic ROIs generated under both pose and lighting effects. What is the relationship between the number of synthetic ROIs and the performance of the FR system?\n\nAnswer: The graph shows that the performance of the FR system improves as the number of synthetic ROIs increases. This is because the more synthetic ROIs that are used to train the classifier, the more likely it is that the classifier will be able to generalize to new data.'

In [49]:
prev_single_turn_total / llava_total_output_tokens

0.29270493741243997

In [50]:
total_output*5 / llava_total_output_tokens

0.7094349738681762